In [1]:
from ultralytics import YOLO
import cv2

# --- Configuration ---
# We are now using a pre-trained YOLO model from Ultralytics.
# The library will automatically download 'yolov8n.pt' the first time.
MODEL_NAME = 'yolov8n.pt' 

# The path to the UI screenshot you want to test
IMAGE_PATH = 'test.png.png'

# --- Load the Model and Image ---
print(f"Loading pre-trained model: {MODEL_NAME}")
try:
    model = YOLO(MODEL_NAME)
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

image = cv2.imread(IMAGE_PATH)
if image is None:
    print(f"Error: Could not read the image at {IMAGE_PATH}")
    exit()

# --- Perform Detection ---
print("Running detection...")
results = model(image)

# --- Draw Boxes on the Image ---
for result in results:
    for box in result.boxes:
        # Get coordinates
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        
        # Get confidence and class name
        confidence = round(float(box.conf[0]), 2)
        class_id = int(box.cls[0])
        class_name = model.names[class_id]

        # Draw rectangle and label
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2) # Red box
        label = f"{class_name}: {confidence}"
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

# --- Display and Save the Result ---
print("Detection complete. Saving result as 'result_pretrained.png' and displaying.")
cv2.imwrite('result_pretrained.png', image)
cv2.imshow('Pre-trained YOLO Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Loading pre-trained model: yolov8n.pt


FileNotFoundError: [Errno 2] No such file or directory: 'test.png.png'

In [1]:
import os
import cv2
from ultralytics import YOLO

# --- CONFIGURATION ---
# 1. Path to your trained YOLO model
MODEL_PATH = 'yolov8n.pt'  # Or whatever you named your custom .pt file

# 2. Path to the folder containing all the RICO screenshots
INPUT_DATASET_DIR = 'images' # <-- IMPORTANT: Change this path

# 3. Path to the folder where you want to save the cropped components
OUTPUT_DIR = './component_dataset'

# --- Counters for unique filenames ---
class_counters = {}

# --- Load Model ---
print(f"Loading custom model from {MODEL_PATH}...")
try:
    model = YOLO(MODEL_PATH)
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# Create the main output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Outputting cropped images to: {OUTPUT_DIR}")

# --- Process Every Image in the Input Folder ---
for image_name in os.listdir(INPUT_DATASET_DIR):
    image_path = os.path.join(INPUT_DATASET_DIR, image_name)
    
    # Check if it's an image file
    if not image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    print(f"\nProcessing: {image_name}")
    
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"  Warning: Could not read image {image_name}, skipping.")
            continue

        # Run detection
        results = model(image)

        # --- Loop Through Detections and Crop ---
        for result in results:
            for box in result.boxes:
                # Get class name
                class_id = int(box.cls[0])
                class_name = model.names[class_id]
                
                # Get bounding box coordinates
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                
                # Crop the original image
                cropped_component = image[y1:y2, x1:x2]

                # --- Create a valid, unique filename ---
                
                # Update counter for this class
                count = class_counters.get(class_name, 0) + 1
                class_counters[class_name] = count
                
                # Create the save directory (e.g., "output/button")
                save_dir = os.path.join(OUTPUT_DIR, class_name)
                os.makedirs(save_dir, exist_ok=True)
                
                # Create the final filename
                save_path = os.path.join(save_dir, f"{class_name}_{count:05d}.png")
                
                # Save the cropped image
                cv2.imwrite(save_path, cropped_component)
                
        print(f"  Found and saved {len(result.boxes)} components.")

    except Exception as e:
        print(f"  Error processing {image_name}: {e}")

print("\n--- Batch Processing Complete ---")
print("Total components saved by class:")
for class_name, count in class_counters.items():
    print(f"  - {class_name}: {count}")

ModuleNotFoundError: No module named 'cv2'